# Контекстные менеджеры и работа с ними

> #### Краткое описание
> * Изучите инструкции `with` и протокол контекстного менеджера
> * Реализуйте класс контекстного менеджера для запроса MongoDB
> * Преобразуйте блок `try...finally` в блок `with` и улучшите читаемость кода.

In [7]:
file = open('output4.txt', mode='r', encoding='utf-8')
file.write('\nPython generation!')
file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'output4.txt'

In [8]:
file = open('output.txt', mode='a', encoding='utf-8')

try:
    file.write('\nPython generation!')
except Exception as error:
    print(f'При записи в файл возникла ошибка: {error}')
finally:
    file.close()

In [9]:
with open('output.txt', mode='a', encoding='utf-8') as file:
    file.write('\nPython generation!')

In [13]:
with open('file.txt', encoding='utf-8') as file, open('output.txt', mode='w', encoding='utf-8') as output:
    for index, line in enumerate(file, 1):
        output.write(f'{index}. {line}')

In [ ]:
with open('file.txt', encoding='utf-8') as file:
    with open('output.txt', mode='w', encoding='utf-8') as output:
        for index, line in enumerate(file, 1):
            output.write(f'{index}. {line}')

---


## `with` Инструкция и протокол контекстного менеджера

Инструкция `with` - это структура потока управления, которая позволяет нам инкапсулировать блоки `try...except...finally` для удобного повторного использования. В результате мы написали более чистый и читаемый код ([PEP 343](https://www.python.org/dev/peps/pep-0343/) | [Python Docs](https://docs.python.org/3/reference/compound_stmts.html#with)).

Оператор `with` поддерживает контекст выполнения, который реализуется с помощью пары методов, выполняемых  перед вводом тела оператора (``__enter__()`) и (2) после выхода из тела оператора (``__exit__()``) ([Источник](https://docs.python.org/3.6/library/stdtypes.html#context-manager-types)).

Базовая структура выглядит следующим образом:
```python
    with context-expression [as var]:
        with_statement_body
```

Для `контекстного выражения` требуется объект, поддерживающий протокол контекстного менеджера, то есть класс, содержащий методы `__enter__()` и `__exit__()`. Мы также можем указать на [контекстный менеджер, написанный с использованием генераторов и декоратора `contextmanager`](https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager).

[В этом блоге](http://arnavk.com/posts/python-context-managers/) дается подробное объяснение специальных методов dunder (двойное подчеркивание).:

> * ``__enter__`` должен возвращать объект, который присваивается переменной после as. По умолчанию это значение равно None и является необязательным. Общий шаблон заключается в том, чтобы возвращать self и сохранять требуемую функциональность в пределах одного класса.
> * ``__exit__` вызывается для исходного объекта контекстного менеджера, а не для объекта, возвращаемого с помощью ``__enter__`.
>* Если в ``__init__` или ``__enter__` возникает ошибка, то блок кода никогда не выполняется и ``__exit__` не вызывается.
> * После ввода блока кода всегда вызывается `__exit__`, даже если в блоке кода возникает исключение.
> * Если `__exit__` возвращает `True`, исключение подавляется.
и __выйти__

Внутри нашего класса мы можем реализовать метод ``__init__()` для настройки нашего объекта, поскольку инструкции не нужно повторять для каждого экземпляра. Для контекстного менеджера базы данных мы можем настроить наше соединение внутри ``__init__()` и возвращать объект или курсор из метода ``__enter__()`.

Переменная, которая следует за ключевым словом `as`, необязательна, но она должна быть включена и использоваться для ссылки на объект, возвращаемый из `__enter__()` внутри нашего `with_statement_body`.

In [1]:
import sys
sys.version

'3.6.1 |Continuum Analytics, Inc.| (default, Mar 22 2017, 19:25:17) \n[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]'

In [14]:
# Давайте разберемся с потоком управления... создадим объект с помощью методов __enter__ и __exit__

class Foo():
    def __init__(self):
        print('__init__ called')
        self.init_var = 0
        
    def __enter__(self):
        print('__enter__ called')
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('__exit__ called')
        if exc_type:
            print(f'exc_type: {exc_type}')
            print(f'exc_value: {exc_value}')
            print(f'exc_traceback: {exc_traceback}')
            
    def add_two(self):
        self.init_var += 2

Протокол контекстных менеджеров состоит всего из двух методов:

__enter__() – вводит контекст и при необходимости возвращает некоторый объект. Значение, возвращаемое этим методом, привязывается к переменной в предложении as оператора with

__exit__() – предоставляет выход из контекста и возвращает логический флаг (тип bool), указывающий на то, следует ли подавлять возбужденное исключение. При возбуждении исключения во время выполнения тела блока with, аргументы содержат тип исключения exc_type, объект исключения exc_value и информацию о трассировке traceback. В противном случае все три аргумента равны None

In [17]:
class Cat:
    def __init__(self, name):
        self.name = name

* exc_type – тип исключения, в данном случае IndexError или другое

* exc_value – объект самого исключения

* traceback – информация о трассировке


In [20]:
class CustomContextManager:
    def __enter__(self):
        print('Вход в контекстный менеджер...')
        return 'Python generation!'

    def __exit__(self, exc_type, exc_value, traceback):
        print('Выход из контекстного менеджера...')
        print(exc_type, exc_value, traceback, sep='\n')

In [21]:
class CustomContextManager:
    def __enter__(self):
        print('Вход в контекстный менеджер...')
        return 'Python generation!'

    def __exit__(self, exc_type, exc_value, traceback):
        print('Выход из контекстного менеджера...')
        if isinstance(exc_value, IndexError):
            print(f'Тип возникшего исключения: {exc_type}')
            print(f'Текст исключения: {exc_value}')
            return True                                 # подавляем возбужденное исключение IndexError

In [ ]:
class CustomContextManager:
    def __init__(self, value):
        self.value = value

    def __enter__(self):
        print('Вход в контекстный менеджер...')
        self.name = 'Кемаль'
        self.breed = 'Британский'
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print('Выход из контекстного менеджера...')
        
    def __repr__(self):
        return f'CustomContextManager(value={repr(self.value)})' 


with CustomContextManager('pygen') as manager:
    print(manager.value)
    print(manager.name)
    print(manager.breed)

In [22]:
with CustomContextManager() as manager:
    print(manager)
    print(manager[100]) 

Вход в контекстный менеджер...
Python generation!
Выход из контекстного менеджера...
Тип возникшего исключения: <class 'IndexError'>
Текст исключения: string index out of range


In [19]:
with Cat("Test") as manager:
    print(manager) # 'Cat' object does not support the context manager protocol

TypeError: 'Cat' object does not support the context manager protocol

In [24]:
with open('output.txt', mode='w', encoding='utf-8') as file:
    print(dir(file))                      # наличие метода __enter__()
    print('__enter__' in dir(file))                      # наличие метода __enter__()
    print('__exit__' in dir(file))                       # наличие метода __exit__()
    file.write('Python generation!')

['_CHUNK_SIZE', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_finalizing', 'buffer', 'close', 'closed', 'detach', 'encoding', 'errors', 'fileno', 'flush', 'isatty', 'line_buffering', 'mode', 'name', 'newlines', 'read', 'readable', 'readline', 'readlines', 'reconfigure', 'seek', 'seekable', 'tell', 'truncate', 'writable', 'write', 'write_through', 'writelines']
True
True


# Использование существующих в Python декораторов

In [25]:
from decimal import Decimal, localcontext

num1 = Decimal('1')
num2 = Decimal('9')

print(num1 / num2)                   # по умолчанию 28 знаков после запятой

with localcontext() as ctx:
    ctx.prec = 5                     # устанавливаем 5 знаков после запятой
    print(num1 / num2)

with localcontext() as ctx:
    ctx.prec = 10                    # устанавливаем 10 знаков после запятой
    print(num1 / num2)

0.1111111111111111111111111111
0.11111
0.1111111111


In [26]:
import os

'''

scandir  возвращает итератор типа ScandirIterator

'''
with os.scandir('.') as entries:
    for entry in entries:
        print(entry.name, '--->', entry.stat().st_size, 'bytes')

file.txt ---> 0 bytes
output.txt ---> 18 bytes
output2.txt ---> 20 bytes
output3.txt ---> 20 bytes
Блокнот_введение_в_контекстные_менеджеры.ipynb ---> 42822 bytes
Блокнот_Примеры_и_магия_декораторов.ipynb ---> 101548 bytes


In [27]:
from tempfile import TemporaryFile

# создавать и удалять временные файлы автоматически
with TemporaryFile(mode='r+') as file:
    file.write('Python generation!')
    file.seek(0)
    content = file.read()
    print(content)

Python generation!


In [ ]:
from threading import Lock


with Lock() as lock:
    # защищенная область
    # смело выполняем любые действия, не думая о гонке потоков

In [3]:
my_object = Foo()

__init__ called


In [4]:
my_object.init_var

0

In [5]:
my_object.add_two()
my_object.init_var

2

In [6]:
# регулярный поток без исключений
with my_object as obj:
    print('inside with statement body')

__enter__ called
inside with statement body
__exit__ called


In [7]:
# к чему мы можем получить доступ в объекте, который возвращается внутри with statement context
with my_object as obj:
    print(obj.init_var)

__enter__ called
2
__exit__ called


In [8]:
# добавление 2 к внутреннему оператору var
with my_object as obj:
    my_object.add_two()
    print(obj.init_var)

__enter__ called
4
__exit__ called


In [9]:
# использование нового экземпляра в контекстном выражении
with Foo() as obj:
    print(obj.init_var)

__init__ called
__enter__ called
0
__exit__ called


# Применение менеджеров (создание)

Для чего применяются:

* открытие и закрытие

* создание и удаление

* изменение данных и возврат к начальным данным

* блокировка и освобождение

* вход и выход

* старт и стоп

* и т.д.

## Пример 1. Контекстный менеджер Trace выводит информацию

In [1]:
class Trace:
    def __enter__(self):
        print('Начало выполнения блока with')

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_value:
            print(f'Во время выполнения блока with было возбуждено исключение {exc_value}')
        print('Конец выполнения блока with')
        return True                           # обрабатываем все типы исключений

In [2]:
with Trace():
    print('Python generation!')

Начало выполнения блока with
Python generation!
Конец выполнения блока with


In [5]:
with Trace():
    print('Python generation!')
    print(1 / 2)

Начало выполнения блока with
Python generation!
0.5
Конец выполнения блока with


In [4]:
with Trace():
    print('Python generation!')
    print(1 / 0)

Начало выполнения блока with
Python generation!
Во время выполнения блока with было возбуждено исключение division by zero
Конец выполнения блока with


## Пример 2. Контекстный менеджер WritableTextFile

In [6]:
class WritableTextFile:
    def __init__(self, path):
        self.path = path

    def __enter__(self):
        self.file = open(self.path, mode='w', encoding='utf-8')
        return self.file

    def __exit__(self, exc_type, exc_value, traceback):
        if self.file:
            self.file.close()

In [7]:
with WritableTextFile('output.txt') as file:
    file.write('Python generation!')

## Пример 3. Контекстный менеджер RedirectedStdout

In [33]:
import sys

class RedirectedStdout:
    def __init__(self, new_output):
        self.new_output = new_output

    def __enter__(self):
        self.standard_output = sys.stdout
        sys.stdout = self.new_output

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.standard_output 

In [34]:
with open('output.txt', mode='w', encoding='utf-8') as file:
    with RedirectedStdout(file):
        print('Python generation!')
    print('Возврат к стандартному потоку вывода')

Возврат к стандартному потоку вывода


# Одноразовые, многоразовые и реентерабельные

Контекстные менеджеры обычно делят на три категории:

* одноразовые

Большинство контекстных менеджеров написаны таким образом, что они могут эффективно использоваться в операторе with только один раз. Такие менеджеры называют одноразовыми, и они должны создаваться заново каждый раз перед использованием. Когда они уже используются или использовались в операторе with – попытка использовать их во второй раз может привести к возбуждению исключения или приведет к неправильной их работе.

* многоразовые

Многоразовый контекстный менеджер – это менеджер, который можно повторно использовать в рамках невложенных операторов with. Примером многоразового контекстного менеджера может служить Timer, позволяющий измерять время выполнения блока кода:

* реентерабельные

Реентерабельный контекстный менеджер – это менеджер, который можно повторно использовать в рамках вложенных операторов with. Примером реентерабельного контекстного менеджера может служить Indenter, который позволяет печатать текст на разных уровнях отступа:

Выглядит как рекурсия контекстных менеджеров.

In [36]:
# создаваться заново каждый раз перед использованием
file = open('output.txt', mode='w', encoding='utf-8')

with file:
    file.write('Python generation!')
    
# with file:
#     file.write('Python generation!') # не переспользуется

In [37]:
# можно повторно использовать в рамках невложенных операторов with
from time import perf_counter

class Timer:
    def __enter__(self):
        self.start = perf_counter()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.elapsed = perf_counter() - self.start

In [38]:
from time import sleep

timer = Timer()

with timer:
    sleep(1.5)
print('Затраченное время:', timer.elapsed)

with timer:
    sleep(0.7)
print('Затраченное время:', timer.elapsed)

with timer:
    sleep(1)
print('Затраченное время:', timer.elapsed)

Затраченное время: 1.5001615000655875
Затраченное время: 0.7000687000108883
Затраченное время: 1.0000844000605866


In [40]:
# Реентерабельный контекстный менеджер - можно повторно использовать в рамках вложенных операторов with
class Indenter:
    def __init__(self):
        self.level = -1

    def __enter__(self):
        self.level += 1 # +1 каждый раз, когда поток выполнения входит в контекст
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.level -= 1 # -1 каждый раз, когда поток выполнения входит в контекст

    def print(self, text):
        print('    ' * self.level + text)

In [41]:
with Indenter() as indent:
    indent.print('python')
    with indent:
        indent.print('beegeek')
        with indent:
            indent.print('stepik')
        indent.print('pygen')
    indent.print('bye-bye')

python
    beegeek
        stepik
    pygen
bye-bye


## Декоратор @contextmanager

В Python создавать контекстные менеджеры можно намного проще с помощью декоратора @contextmanager из модуля contextlib. Декоратор @contextmanager позволяет создать контекстный менеджер на основе функции, автоматически предоставляя оба требуемых метода __enter__() и __exit__().

In [42]:
class CustomContextManager:
    def __enter__(self):
        print('Вход в контекстный менеджер...')
        return 'Python generation!'

    def __exit__(self, exc_type, exc_value, traceback):
        print('Выход из контекстного менеджера...')

In [47]:
from contextlib import contextmanager

@contextmanager
def custom_context_manager():
    print('Вход в контекстный менеджер...')
    yield 'Python generation!'
    print('Выход из контекстного менеджера...')

In [48]:
with custom_context_manager() as manager:
    print(manager)

Вход в контекстный менеджер...
Python generation!
Выход из контекстного менеджера...


In [49]:
from contextlib import contextmanager

@contextmanager
def custom_context_manager():
    print('Вход в контекстный менеджер...')
    try:
        yield 'Python generation!'
    except IndexError as error:
        print(f'Тип возбужденного исключения: {type(error)}')
        print(f'Текст исключения: {error}')
    except:
        raise           # если исключение не планируется подавлять, оно должно быть возбуждено повторно
    finally:
        print('Выход из контекстного менеджера...')

In [50]:
with custom_context_manager() as manager:
    print(manager)
    print(manager[100]) 

Вход в контекстный менеджер...
Python generation!
Тип возбужденного исключения: <class 'IndexError'>
Текст исключения: string index out of range
Выход из контекстного менеджера...


## Применение с  БД

In [ ]:
from contextlib import contextmanager
import psycopg2

@contextmanager
def db_connection(host, database, user, password):
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )
    try:
        cur = conn.cursor()
        yield cur
        conn.commit()
    except Exception as e:
        conn.rollback()
        raise e
    finally:
        conn.close()

with db_connection('localhost', 'postgres', 'postgres', 'postgres') as conn:
    cur = conn.cursor()
    cur.execute("INSERT INTO users (name, email) VALUES (%s, %s)", ('John Doe', 'john@example.com'))
    cur.execute("SELECT * FROM users WHERE email = %s", ('john@example.com',))
    row = cur.fetchone()
    print(row)

# После выхода из блока 'with' соединение автоматически закрыто


In [ ]:
import asyncio
from contextlib import contextmanager, AsyncExitStack
import psycopg2

@contextmanager
def asyncio_db_connection(host, database, user, password):
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )
    try:
        yield conn
    finally:
        conn.close()

async def main():
    async with AsyncExitStack() as stack:
        db_conn = await stack.enter_async_context(asyncio_db_connection('localhost', 'postgres', 'postgres', 'postgres'))
        async with db_conn.cursor() as cur:
            await cur.execute("SELECT * FROM users WHERE id = %s", (1,))
            row = await cur.fetchone()
            print(row)
    
asyncio.run(main())

# После выхода из блока 'with' соединение автоматически закрыто


## Когда использовать контекстные менеджеры

Дэйв Брондсема выступил с [замечательным докладом о декораторах и контекстных менеджерах](https://www.youtube.com/watch?v=cSbD5SKwak0) на конференции PyCon 2012. [Он упомянул] (https://youtu.be/cSbD5SKwak0?t=13m15s), что нам следует использовать контекстные менеджеры, когда мы видим в нашем коде любой из следующих шаблонов:
* `Открыть` - `Закрыть` (смотрите пример ниже)
* `Заблокировать` - `Разблокировать`
* `Изменить` - `Сбросить`
* `Войти` - `Выйти`
* `Начать` - `Остановить`

Арнав Кхаре подробно описывает множество отличных примеров использования [контекстных менеджеров в реальном мире] (http://arnavk.com/posts/python-context-managers/) и предоставляет начальный код для каждого примера.

---

---

#### Дополнительные ресурсы

* [Jeff Knupp - Python with Context Managers](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/)
* [*Python Tips* - Context Managers](http://book.pythontips.com/en/latest/context_managers.html)
* [StackOverflow *(Praise Be)* discussion](http://stackoverflow.com/questions/3693771/trying-to-understand-python-with-statement-and-context-managers)
* [Arnav Khare - Python in the real world: Context Managers](http://arnavk.com/posts/python-context-managers/)
* [Context Managers: Advanced Techniques YouTube video](https://www.youtube.com/watch?v=ORo1-sXmvGg&t=1822s)
* [Dave Brondsema - Decorators and Context Managers](https://www.youtube.com/watch?v=cSbD5SKwak0) (PyCon 2012)